In [5]:
import requests
import csv
import pandas as pd

def get_hotel_info(api_key, hotel_name):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={hotel_name}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if 'results' in data and data['results']:
        first_result = data['results'][0]
        hotel_info = {
            'name': first_result['name'],
            'address': first_result['formatted_address'],
            'rating': first_result.get('rating', 'N/A'),
            'place_id': first_result['place_id'],
            'geometry': first_result['geometry']['location'],
        }
        return hotel_info
    else:
        print(f"No results found for {hotel_name}.")
        return None

def count_nearby_places(api_key, location, radius=5000, types=None):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location['lat']},{location['lng']}&radius={radius}&key={api_key}"
    if types:
        url += f"&types={','.join(types)}"

    response = requests.get(url)
    data = response.json()

    type_counts = {place_type: 0 for place_type in types}

    if 'results' in data:
        for place in data['results']:
            place_types = place.get('types', [])
            for place_type in place_types:
                if place_type in type_counts:
                    type_counts[place_type] += 1

    return type_counts

hotel_data = pd.read_csv('agoda_booking_combined.csv')

# API key
api_key = ""

# Specify types of places to search for
types = ['restaurant', 'shopping_mall', 'tourist_attraction', 'cafe', 'bar', 'supermarket', 'park', 'point_of_interest', 'lodging', 'museum']

# Create a CSV file and write the data
with open('google_place_api_summary.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['Original Hotel Name'] + ['Hotel Name'] + ['Geometry'] + types)
    
    for hotel_name in hotel_data['hotel_name']:
        hotel_info = get_hotel_info(api_key, hotel_name)
        if hotel_info:
            location = hotel_info['geometry']
            type_counts = count_nearby_places(api_key, location, types=types)
            writer.writerow([hotel_name] + [hotel_info['name']] + [location] + [type_counts.get(place_type, 0) for place_type in types])
        else:
            writer.writerow([hotel_name] + ['No information found'] + ['N/A'] + [0] * len(types))

print("CSV file created successfully.")

No results found for Metro PHX STUDIO QUEEN sleeps 4 HEATED POOL.
No results found for 3 HOMESTAY At Boston In Dorchester or JFK-UMASS or South Boston.
No results found for Hotel AKA Back Bay Boston.
No results found for Hotel AKA Back Bay Boston.
No results found for The Boxer.
No results found for Comfy Beacon Hill Studio Great for Work Travel #7.
No results found for Hotel Brexton, Trademark Collection by Wyndham.
No results found for Brand new bedroom with Tv next JHU.
No results found for GORGEOUS HOUSE NEXT JHU.
No results found for Detroit Southfield.
No results found for Comfort Inn Downtown.
No results found for Lovely 1-bedroom apartment w/ Pool + Wi-Fi.
No results found for The Summit Hotel.
No results found for Classy Studio Downtown Near Hard Rock Casino.
No results found for Huge 2 Bedroom House in the Heart of Northside!.
No results found for The Paramount Hotel.
No results found for Charming Alabama Retreat with Patio and Pool Access!.
No results found for Free Heated P